### Шаг 1: Генерация проблемных данных

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Types_Lab").getOrCreate()

data = [
    ("1", "2023-01-01", "100", None),           # Хорошая строка
    ("2", "01/02/2023", "ERROR", 50.0),         # Плохая дата, плохое число
    ("3", "2023-02-01", None, float('nan')),    # Null в сумме, NaN в скидке
    ("4", None, "200", 0.0)                     # Null в дате
]

# Изначально читаем все как строки (как часто бывает с CSV)
columns = ["id", "event_date", "amount_str", "discount"]

df_dirty = spark.createDataFrame(data, schema=columns)
df_dirty.show()
df_dirty.printSchema() # Все string (кроме discount, который double)

+---+----------+----------+--------+
| id|event_date|amount_str|discount|
+---+----------+----------+--------+
|  1|2023-01-01|       100|    NULL|
|  2|01/02/2023|     ERROR|    50.0|
|  3|2023-02-01|      NULL|     NaN|
|  4|      NULL|       200|     0.0|
+---+----------+----------+--------+

root
 |-- id: string (nullable = true)
 |-- event_date: string (nullable = true)
 |-- amount_str: string (nullable = true)
 |-- discount: double (nullable = true)



### Шаг 2: Приведение типов (Casting) и Парсинг дат
Мы используем .cast() для смены типа и to_date() для дат.

In [2]:
# Чистим данные
df_typed = df_dirty \
    .withColumn("id", F.col("id").cast("integer")) \
    .withColumn("amount", F.col("amount_str").cast("double")) \
    .withColumn("clean_date", F.to_date(F.col("event_date"), "yyyy-MM-dd")) \
    .withColumn("correct_date", 
                F.coalesce(
                    F.to_date(F.col("event_date"), "yyyy-MM-dd"), # Попытка 1: формат ISO
                    F.to_date(F.col("event_date"), "dd/MM/yyyy")  # Попытка 2: формат EU
                )
    )

df_typed.show()

# Обратите внимание:
# 1. "ERROR" при касте в double превратился в NULL (безопасно).
# 2. Неправильный формат даты превратился в NULL в колонке clean_date, 
#    но восстановился в correct_date благодаря coalesce.

+---+----------+----------+--------+------+----------+------------+
| id|event_date|amount_str|discount|amount|clean_date|correct_date|
+---+----------+----------+--------+------+----------+------------+
|  1|2023-01-01|       100|    NULL| 100.0|2023-01-01|  2023-01-01|
|  2|01/02/2023|     ERROR|    50.0|  NULL|      NULL|  2023-02-01|
|  3|2023-02-01|      NULL|     NaN|  NULL|2023-02-01|  2023-02-01|
|  4|      NULL|       200|     0.0| 200.0|      NULL|        NULL|
+---+----------+----------+--------+------+----------+------------+



### Шаг 3: Обработка Null-значений (df.na)
Теперь у нас много Null'ов. Как с ними работать?

#### Стратегия 1: Заполнение (Fill) Заменяем Null на константу (0 для чисел, "Unknown" для строк).

In [3]:
# Заменяем Null в amount на 0, а в discount на 0
df_filled = df_typed.na.fill(value=0, subset=["amount", "discount"])
df_filled.show()

+---+----------+----------+--------+------+----------+------------+
| id|event_date|amount_str|discount|amount|clean_date|correct_date|
+---+----------+----------+--------+------+----------+------------+
|  1|2023-01-01|       100|     0.0| 100.0|2023-01-01|  2023-01-01|
|  2|01/02/2023|     ERROR|    50.0|   0.0|      NULL|  2023-02-01|
|  3|2023-02-01|      NULL|     0.0|   0.0|2023-02-01|  2023-02-01|
|  4|      NULL|       200|     0.0| 200.0|      NULL|        NULL|
+---+----------+----------+--------+------+----------+------------+



#### Стратегия 2: Удаление (Drop) Удаляем строки, где есть Null в важных колонках.

In [4]:
# Удалить строку, если Null есть ХОТЯ БЫ В ОДНОЙ из указанных колонок
df_dropped = df_typed.na.drop(subset=["amount", "clean_date"])
df_dropped.show()

+---+----------+----------+--------+------+----------+------------+
| id|event_date|amount_str|discount|amount|clean_date|correct_date|
+---+----------+----------+--------+------+----------+------------+
|  1|2023-01-01|       100|    NULL| 100.0|2023-01-01|  2023-01-01|
+---+----------+----------+--------+------+----------+------------+



#### Стратегия 3: SQL Coalesce (Приоритет) Функция F.coalesce(c1, c2, c3) берет первое не-Null значение из списка.

In [5]:
# Логика: Если amount есть - берем его. Если нет - берем 0.
df_calc = df_typed.withColumn("final_amount", F.coalesce(F.col("amount"), F.lit(0)))
df_calc.select("amount", "final_amount").show()

+------+------------+
|amount|final_amount|
+------+------------+
| 100.0|       100.0|
|  NULL|         0.0|
|  NULL|         0.0|
| 200.0|       200.0|
+------+------------+



Анти-паттерны и Ошибки  
🚫 Ошибка 1: Сравнение с Null через ==  
Неверно: df.filter(F.col("amount") == None) или == "null".

Почему: В SQL/Spark Null == Null дает Null (False).

Как надо: .filter(F.col("amount").isNull()) или .isNotNull().

🚫 Ошибка 2: Арифметика без страховки  
Сценарий: df.withColumn("total", col("price") + col("tax")).

Проблема: Если tax равен Null (например, налог не указан), то total тоже станет Null, и мы потеряем цену.

Как надо: F.coalesce(col("price"), F.lit(0)) + F.coalesce(col("tax"), F.lit(0)).